### To setup the environment for downloading the dataset and the YOLOv5 model.

In [92]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt
!pip install roboflow

import torch

from roboflow import Roboflow
rf = Roboflow(api_key="<API_KEY")
project = rf.workspace("yolov7-twb3s").project("step2-v1_2-mud-illu-6-beixinyao-vwcvx")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Step2-v1_2-mud-illu-6-beixinyao-2 in yolov5pytorch:: 100%|████████████████████████████████████████| 5270/5270 [00:11<00:00, 477.66it/s]


### Train on polygons

In [ ]:
!python yolov5/segment/train.py --data Step2-v1_2-mud-illu-6-beixinyao-2/data.yaml --img 320 --epochs 100 -- batch 128 --name trained-seg

### Test model on bounding boxes

In [ ]:
!python yolov5/segment/predict.py --save-txt --weights yolov5/train-seg/trained-seg/weights/best.pt --img 320  --source 'Step2-v1_2-mud-illu-6-beixinyao-2/test/images/*.jpg'

**Convert both the predicted polygons and the polygon test set from polygon to bounding box coordinates.**

In [1]:
import numpy as np
import os
from sklearn.metrics import pairwise_distances
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.metrics import precision_recall_curve
from torchmetrics.classification import PrecisionRecallCurve
import cv2


def convert_to_bounding_box(segmentation_label):
    x_coord, y_coord = [], []
    for i in range(0, len(segmentation_label), 2):
        x_coord.append(segmentation_label[i])
        y_coordinates.append(segmentation_label[i+1])
    return convert_to_yolov5(min(x_coord), min(y_coord), max(x_coord), max(y_coord))

def convert_to_yolov5(xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / 2
    y_center = (ymin + ymax) / 2
    width = xmax - xmin
    height = ymax - ymin
    return np.array((x_center, y_center, width, height))


def from_xmin_ymin_xmax_ymax_to_yolov5(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()
            for i, line in enumerate(lines):
                values = line.split()
                class_label = values[0]
                polygon = [float(val) for val in values[1:]]
                bbox = convert_to_bounding_box(polygon)
                lines[i] = f"{class_label} {' '.join(map(str, bbox))}\n"
            with open(file_path, 'w') as file:
                file.writelines(lines)
                
from_xmin_ymin_xmax_ymax_to_yolov5('yolov5/runs/predict-seg/exp/labels')
from_xmin_ymin_xmax_ymax_to_yolov5('Step2-v1_2-mud-illu-6-beixinyao-2/test/labels')


### Compute metrics
**Run this cell tu get the Precision, Recall and mAP**

In [2]:
def iou(box1, box2):
    """
    Calculates the Intersection over Union (IoU) of two bounding boxes.
    box1 and box2 are arrays of length 4: [x_center, y_center, width, height].
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate intersection area
    x_left = max(x1 - w1/2, x2 - w2/2)
    y_top = max(y1 - h1/2, y2 - h2/2)
    x_right = min(x1 + w1/2, x2 + w2/2)
    y_bottom = min(y1 + h1/2, y2 + h2/2)
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    # Calculate union area
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou


def match_boxes(boxes1, boxes2):
    """
    Matches boxes in boxes1 to boxes in boxes2 based on IoU.
    boxes1 and boxes2 are arrays of shape (n_boxes, 5) where the first
    element of each row is the class (0 or 1) and the remaining 4 elements
    are [x_center, y_center, width, height].
    Returns a list of tuples, where each tuple contains the indices of the
    matched boxes in boxes1 and boxes2, or None if no match was found.
    """
    n_boxes1 = boxes1.shape[0]
    n_boxes2 = boxes2.shape[0]
    
    # Calculate IoU matrix
    iou_matrix = np.zeros((n_boxes1, n_boxes2))
    for i in range(n_boxes1):
        for j in range(n_boxes2):
            iou_matrix[i,j] = iou(boxes1[i], boxes2[j])
    
    # Find best matches
    matches = []
    
    for i in range(n_boxes1):
        j = np.argmax(iou_matrix[i])
        if iou_matrix[i,j] > 0:
            # Check if box j is already matched to another box
            matched_boxes = [m[1] for m in matches if m is not None]
            if j not in matched_boxes:
                matches.append((i, j))
            else:
                # If box j is already matched, find the best unmatched box
                ious = iou_matrix[i,:]
                ious[matched_boxes] = 0
                j2 = np.argmax(ious)
                if iou_matrix[i,j2] >= 0.5:
                    matches.append((i, j2))
    return matches

def compute_recall(pred_boxes, pred_classes, pred_confidence, gt_boxes, gt_classes, iou_threshold=0.5, confidence_threshold=0.5):
    # Filter predicted boxes based on confidence score
    positive_mask = pred_confidence >= confidence_threshold
    pred_boxes = pred_boxes[positive_mask]
    pred_classes = pred_classes[positive_mask]

    # Compute IoU between predicted boxes and ground truth boxes
    iou_matrix = pairwise_distances(pred_boxes, gt_boxes, metric=iou)
    matches = iou_matrix > iou_threshold

    # Count true positives for each ground truth box
    tp_counts = np.sum(matches, axis=0)
    tp_mask = tp_counts > 0

    # Compute recall
    recall = np.sum(tp_mask) / gt_boxes.shape[0]

    return recall

def compute_precision(pred_boxes, pred_classes, pred_confidence, gt_boxes, gt_classes, iou_threshold=0.5, confidence_threshold=0.5):
    # Filter predicted boxes based on confidence score
    positive_mask = pred_confidence >= confidence_threshold
    pred_boxes = pred_boxes[positive_mask]
    pred_classes = pred_classes[positive_mask]

    # Compute IoU between predicted boxes and ground truth boxes
    iou_matrix = pairwise_distances(pred_boxes, gt_boxes, metric=iou)
    matches = iou_matrix > iou_threshold

    # Count true positives for each predicted box
    tp_counts = np.sum(matches, axis=1)
    tp_mask = tp_counts > 0

    # Compute precision
    precision = np.sum(tp_mask) / pred_boxes.shape[0]

    return precision

# folder path containing txt files
folder_path = "yolov5/runs/predict-seg/exp/labels/"
folder_path_test = "Step2-v1_2-mud-illu-6-beixinyao-2/test/labels/"
mAP, p, r = 0, 0, 0
how_many = 0


for filename in os.listdir(folder_path):
    how_many += 1
    preds_ann = {'boxes':[], 'scores':[], 'labels':[]}
    with open(os.path.join(folder_path, filename), "r") as file:
        for line in file:
            values = [float(value) for value in line.strip().split()]
            bbox = convert_to_bounding_box(values[1:-1])
            class_obj = int(values[0])
            confidence = values[-1]
            preds_ann['boxes'].append(bbox)
            preds_ann['scores'].append(confidence)
            preds_ann['labels'].append(class_obj)
    preds = [dict(boxes=preds_ann['boxes'], scores=preds_ann['scores'], labels=preds_ann['labels'])]
    
    target_ann = {'boxes':[], 'labels':[]}
    with open(os.path.join(folder_path_test, filename), "r") as file:
            for line in file:
                values = [float(value) for value in line.strip().split()]
                class_obj = int(values[0])
                target_ann['boxes'].append(values[1:])
                target_ann['labels'].append(class_obj)    
    target = [dict(boxes=target_ann['boxes'], labels=target_ann['labels'])]
    
    # box matching
    matches = match_boxes(np.array(target[0]['boxes']), np.array(preds[0]['boxes']))
    
    # swap
    for i in range(len(matches)):
        preds[0]['boxes'][matches[i][1]] = target[0]['boxes'][matches[i][0]]
        preds[0]['labels'][matches[i][1]] = target[0]['labels'][matches[i][0]]
    
    preds[0]['boxes'] = torch.from_numpy(np.array(preds[0]['boxes']))
    preds[0]['scores'] = torch.from_numpy(np.array(preds[0]['scores']))
    preds[0]['labels'] = torch.from_numpy(np.array(preds[0]['labels']))

    target[0]['boxes'] = torch.from_numpy(np.array(target[0]['boxes']))
    target[0]['labels'] = torch.from_numpy(np.array(target[0]['labels']))
    
    # compute metrics
    metric = MeanAveragePrecision(box_format='cxcywh')
    metric.update(preds, target)
    mAP += metric.compute()['map'].item()
    r += compute_recall(preds[0]['boxes'], preds[0]['labels'], preds[0]['scores'], target[0]['boxes'], target[0]['labels'])
    p += compute_precision(preds[0]['boxes'], preds[0]['labels'], preds[0]['scores'], target[0]['boxes'], target[0]['labels'])
                       
print('Precision: {}, Recall: {}, mAP: {}'.format(p/how_many, r/how_many, mAP/how_many))

### Train on bounding boxes

In [ ]:
!python yolov5/train.py --data Step2-v1_2-mud-illu-6-beixinyao-2/data.yaml --img 320 --epochs 100 -- batch 128 --name trained-od

### Computes metrics

In [62]:
!python yolov5/val.py --weights OD/trained_on_bbox/train/bestOD.pt --img 320 --data Step2-v1_2-mud-illu-6-beixinyao-2/data.yaml

val: data=Step2-v1_2-mud-illu-6-beixinyao-2/data.yaml, weights=['OD/trained_on_bbox/train/bestOD.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-105-g226a5e4 Python-3.9.13 torch-1.13.1+cu117 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/vali
val: New cache created: /home/pptr/Desktop/roboflow/Step2-v1_2-mud-illu-6-beixinyao-2/valid/labels.cache
                 Class     Images  Instances          P          R      mAP50   
                   all         51        939      0.977      0.947      0.981      0.773
                  coal         51        308      0.986      0.977      0.994      0.806
     